In [1]:
import os
import shutil
import glob
import pdfplumber
import pandas as pd
import smtplib
import keyring

from email import encoders
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage

In [ ]:
#Ler e Ajustar nomes dos PDF Originais
path = './Originais'
for file in glob.iglob(path + '\*.pdf'):
    pdf = pdfplumber.open(file)
    pdf_text = pdf.pages[0].extract_text()
    pdf.close()

    tpDoc = pdf_text[0:15].split()[0]
    
    if tpDoc == 'Documento': #DAS
        pos = pdf_text.find('CNPJ Razão Social')
        cnpj = pdf_text[pos: pos+36].split()[-1]
        cnpj = cnpj.replace('/','').replace('-','').replace('.','').zfill(14)
        newName = 'DAS ' + cnpj + '.pdf'   

    if tpDoc == 'Extrato': #PGDAS-R Extrato
        for pg in pdf.pages:
            pdf_text = pg.extract_text()
            pos = pdf_text.find('CNPJ Estabelecimento: ')
            if pos > -1:
                cnpj = pdf_text[pos: pos+40].split(': ')[1]
                cnpj = cnpj.replace('/','').replace('-','').replace('.','').zfill(14)
                newName = 'PGDAS-D ' + cnpj + '.pdf'

    if tpDoc == 'Programa': #PGDAS-R Declaratorio
        pos = pdf_text.find('CNPJ Matriz:')
        cnpj = pdf_text[pos: pos+31].split()[-1]
        cnpj = cnpj.replace('/','').replace('-','').replace('.','').zfill(14)
        newName = 'PGDAS-D ' + cnpj + '.pdf'   

    # print('Renomeando: {} para: {}' .format(file, os.path.join(path , newName)))
    newfile = os.path.join(path , newName)
    if os.path.isfile(newfile):
        print('{} já existe!' .format(newName))
    else:
        os.rename(file, newfile)   

In [ ]:
def EnviarEmail(cnpj, empresa, para):
    try:
        msg = MIMEMultipart()

        msg['From'] = 'NotaFiscal@unicacontabil.com' 
        # msg['To'] = 'HenriqueCaziniFerreira@gmail.com'
        msg['To'] = para
        # msg['Cc'] = 'fiscal@unicacontabil.com'
        msg['Subject'] = 'DAS (SIMPLES NACIONAL) {} - {}' .format(nmApuracao, empresa)

        body = """
        
Bom Dia,

Segue anexo DAS referente ao mês Anterior


Atenciosamente
Única Contábil
Departamento Fiscal
(62) 3212-1200 | (62) 98228-0118 
"""

        msg.attach(MIMEText(body, 'plain'))
    
        #Assinatura Imagem
        jpg = 'C:/Temp/Venv Py/UnicaContabil/Assinatura Luan.jpg'
        fp = open(jpg, 'rb')
        msg_img = MIMEImage(fp.read())
        fp.close()
        msg_img.add_header('Content-ID', '<{}>'.format("Assinatura.jpg"))
        msg.attach(msg_img)

        #Anexar DAS
        file = './Originais/DAS {}.pdf'.format(cnpj)
        if os.path.isfile(file):
            attachment = open(file,'rb')
            part = MIMEBase('application', 'octet-stream')
            part.set_payload((attachment).read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', "attachment; filename= DAS.pdf")
            msg.attach(part)
            attachment.close()

        #Anexar PGDAS
        file = './Originais/PGDAS-D {}.pdf'.format(cnpj)
        if os.path.isfile(file):
            attachment = open(file,'rb')
            part = MIMEBase('application', 'octet-stream')
            part.set_payload((attachment).read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', "attachment; filename=PGDAS-D.pdf")
            msg.attach(part)
            attachment.close()

        #Enviar Email
        server = smtplib.SMTP('smtp.gmail.com: 587')
        server.starttls()
        server.login('NotaFiscal@unicacontabil.com', keyring.get_password('.py', 'NotaFiscal'))
        text = msg.as_string()
        server.sendmail(msg['From'], para.split(';'), text)
        server.quit()
        print('{} Ok!'.format(empresa))
    except:
        print("Erro ao enviar para {}".format(empresa))

In [ ]:
def MoverArquivos(cnpj, pasta):
    file = './Originais/DAS {}.pdf'.format(cnpj)
    if os.path.isfile(file):
        shutil.move(file, os.path.join(pasta, 'DAS.pdf'))

    file = './Originais/PGDAS-D {}.pdf'.format(cnpj)
    if os.path.isfile(file):
        shutil.move(file, os.path.join(pasta, 'PGDAS-D.pdf'))


In [ ]:
# Carregar Dados Planilha
arq = 'ENVIAR SIMPLES NACIONAL.xlsx'

df = pd.read_excel(arq, header=0)

filtro = [3011114000125, 8301978000103, 35501843000156, 43664673000159, 37853161000138]
df2 = df[df['CNPJ'].isin(filtro)]

# df2 = df.head(10)

df2.set_index('CÓDIGO', inplace=True)
df2.fillna('-', inplace=True)

In [ ]:
pathDe = './Originais'
pathPara = './Arquivamento'

apuracao = '2022_06'
nmApuracao = 'Junho/2022'

for index, row in df2.iterrows(): 
    # Cria a Pasta da Empresa
    client = str(index) + ' - ' + str(row['EMPRESA']).replace('/', ' ')
    pasta = os.path.join(pathPara, client)
    if not os.path.isdir(pasta): 
        os.mkdir(pasta) 
    
    # Cria a Pasta da Data de Apuração
    pasta = os.path.join(pathPara, client, apuracao)
    if not os.path.isdir(pasta): 
        os.mkdir(pasta) 

    # Lista de Email
    # lEmail =[]
    # lEmail.append(row['EMAIL'])
    # lEmail.append(row['EMAIL.1'])
    # lEmail.append(row['EMAIL.2'])
    # lEmail.append(row['EMAIL.3'])
    # lEmail.append(row['EMAIL.4'])
    # lEmail.append(row['EMAIL.5'])
    # lEmail.append(row['EMAIL.6'])

    lEmail =[]
    lEmail.append('HenriqueCaziniFerreira@gmail.com')
    lEmail.append('Evelyn@unicacontabil.com')
    lEmail.append('Evy.lima@hotmail.com')

    cnpj = str(row['CNPJ']).zfill(14)

    lista = list(filter(lambda x: x != '-', lEmail))
    para = ';'.join(lista)

    #Enviar Email
    EnviarEmail(cnpj, row['EMPRESA'], para)
    
    #Mover Arquivo
    MoverArquivos(cnpj, pasta)


In [ ]:
#Assinatura Imagem
file = '../Assinatura Luan.jpg'

# file = './Originais/DAS {}.pdf'.format(cnpj)
print(os.path.isfile(file))


In [ ]:
# Teste da construção do E-mail

msg = MIMEMultipart()

msg['From'] = 'NotaFiscal@unicacontabil.com' 
msg['To'] = 'HenriqueCaziniFerreira@gmail.com'
msg['Subject'] = 'DAS (SIMPLES NACIONAL) {} - {}'

body = """
        
Bom Dia,

Segue anexo DAS referente ao mês Anterior

Atenciosamente
Única Contábil
Departamento Fiscal
(62) 3212-1200 | (62) 98228-0118 
"""

msg.attach(MIMEText(body, 'plain'))

#Assinatura Imagem
jpg = '../Assinatura Luan.jpg'
fp = open(jpg, 'rb')
msg_img = MIMEImage(fp.read())
fp.close()
msg_img.add_header('Content-ID', '<{}>'.format("Assinatura.jpg"))
msg.attach(msg_img)


print(msg.as_string())

In [52]:
import re

# Pegar Dados Tabela
file = './Originais/PGDAS-D 08301978000103.pdf'
pdf = pdfplumber.open(file)
pdf_text = pdf.pages[0].extract_text()
pdf.close()

tpDoc = pdf_text[0:15].split()[0]

mi = pdf_text.find('2.2.1) Mercado Interno')
me = pdf_text.find('2.2.2) Mercado Externo')

tbMI = pdf_text[mi + 22:me].replace('\n', '').replace(' ','')
print(re.split('^{2}$/{4}$',tbMI))
# print(pdf_text)

error: nothing to repeat at position 1